In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indic-tts-deepfake-challenge/sample.csv


In [2]:
df = pd.read_csv('/kaggle/input/indic-tts-deepfake-challenge/sample.csv')
df.head()

,id,is_tts
0,ASM_F_ANGER_00109,0.170354
1,ASM_F_ANGER_00127,0.241722
2,ASM_F_ANGER_00386,0.784936
3,ASM_F_ANGER_00103,0.843566
4,ASM_F_ANGER_00434,0.759560


In [3]:
import json
import torch
print(torch.__version__)
import torchaudio

2.5.1+cu121


In [4]:
import librosa
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2Model

<h1>Load the Dataset</h1>

In [5]:
from datasets import load_dataset
ds = load_dataset("SherryT997/IndicTTS-Deepfake-Challenge-Data")
print(ds)

README.md:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

train-00000-of-00035.parquet:   0%|          | 0.00/453M [00:00<?, ?B/s]

train-00001-of-00035.parquet:   0%|          | 0.00/461M [00:00<?, ?B/s]

...

test-00003-of-00004.parquet:   0%|          | 0.00/291M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2635 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/35 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'language', 'is_tts', 'audio'],
        num_rows: 31102
    })
    test: Dataset({
        features: ['text', 'id', 'language', 'is_tts', 'audio'],
        num_rows: 2635
    })
})


In [6]:
train_ds = ds["train"]
test_ds = ds["test"]

In [7]:
train_ds = train_ds.shuffle(seed=37).select(range(10000))

In [8]:
train_data = pd.DataFrame({'id': train_ds['id'], 'language': train_ds['language'], 'is_tts': train_ds['is_tts']})

<h1>Data Preparation</h1>

Setup device agnostics

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Feature Extraction

In [10]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

In [11]:
def prepare_sample(batch):
    # extract the audio data
    audio = batch["audio"]["array"] # waveform
    expected_length = 32000  # 2 sec of audio (32,000 samples)
    # The audio is assumed to have a sampling rate of 16,000 Hz (i.e., 16,000 samples per second)

    # Trim or pad to 2 sec
    if len(audio) < expected_length:
        audio = np.pad(audio, (0, expected_length - len(audio)), mode='constant')
    else:
        audio = audio[:expected_length]

    # audio to feature representation (specific to HuBERT)
    inputs = feature_extractor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    batch["input_values"] = inputs.input_values[0]  # Extract features

    # Convert label (0 (real speech) or 1 (synthetic speech)) into a tensor
    batch["labels"] = torch.tensor(batch["is_tts"], dtype=torch.float)

    return batch

In [12]:
# Apply preprocessing
sampled_data = train_ds.map(
    prepare_sample, 
    remove_columns=["audio", "text", "id", "language", "is_tts"]
)

# Split dataset
data = sampled_data.train_test_split(test_size=0.1, shuffle=True, seed=42)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [13]:
!pip install evaluate
import torch
from typing import List, Dict, Union
from dataclasses import dataclass
from evaluate import load
from transformers import DataCollatorWithPadding

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00


In [14]:
@dataclass  # decorator automatically generates an __init__ method for the class.
class DataCollatorWithPadding:
    """
    Data collator that dynamically pads inputs but keeps labels as-is.
    """
    processor: feature_extractor  # Use full processor for consistency
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Extract input features (waveforms)
        input_features = [{"input_values": feature["input_values"]} for feature in features]

        # Apply padding using processor
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Convert labels to tensor
        batch["labels"] = torch.tensor([feature["labels"] for feature in features], dtype=torch.long)

        return batch


data_collator = DataCollatorWithPadding(feature_extractor, padding=True)

<h1>Model</h1>

In [15]:
from transformers import AutoModelForAudioClassification,AutoFeatureExtractor

In [16]:
# Load DistilHuBERT model & processor
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

model = AutoModelForAudioClassification.from_pretrained(model_id, num_labels=2)  # Binary classification

preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model.to(device)

HubertForSequenceClassification(
  (hubert): HubertModel(
    (feature_extractor): HubertFeatureEncoder(
      (conv_layers): ModuleList(
        (0): HubertGroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x HubertNoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): HubertFeatureProjection(
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): HubertEncoder(
      (pos_conv_embed): HubertPositionalConvEmbedding(

In [18]:
model.freeze_feature_encoder()

<h1>Training</h1>

In [19]:
from transformers import TrainingArguments
from transformers import Trainer
from scipy.special import softmax

In [20]:
# Load evaluation metrics
accuracy_metric = load("accuracy")
precision_metric = load("precision")
recall_metric = load("recall")
f1_metric = load("f1")
roc_auc_metric = load("roc_auc")

In [21]:
def compute_metrics(pred):
    # Extract logits from model predictions
    pred_logits = pred.predictions
    
    # Convert logits to probabilities using softmax
    pred_probs = softmax(pred_logits, axis=-1)[:, 1]  # Probability of class 1 (synthetic speech)

    # Get true labels
    labels = pred.label_ids

    # Compute metrics
    accuracy = accuracy_metric.compute(predictions=pred_probs.round(), references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["precision"]
    recall = recall_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["recall"]
    f1 = f1_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["f1"]
    roc_auc = roc_auc_metric.compute(prediction_scores=pred_probs, references=labels)["roc_auc"]

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc  # Important for evaluating probability-based outputs
    }

In [22]:
print(torch.cuda.is_available()) 
print(torch.cuda.device_count())  
print(torch.cuda.current_device())  
print(torch.cuda.get_device_name(0))

True
1
0
Tesla P100-PCIE-16GB


In [23]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use only GPU 0
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_DEBUG"] = "WARN"

training_args = TrainingArguments(
    output_dir="AsrTaskModel",
    per_device_train_batch_size=4,  
    per_device_eval_batch_size=4,
    eval_strategy="steps",
    num_train_epochs=7,
    fp16=False,  
    gradient_checkpointing=True,
    save_steps=1000,
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4,
    weight_decay=0.005,
    warmup_steps=1000,
    save_total_limit=2,
    load_best_model_at_end=True,
    save_strategy="steps",
    report_to="none"
)

In [24]:
# Define trainer
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    processing_class=feature_extractor,  # Only needed for collator
)

In [25]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
500,0.589000,0.343438,0.845000,0.876485,0.781780,0.826428,0.932536
1000,0.395000,0.270101,0.888000,0.914747,0.841102,0.876380,0.966603
1500,0.358300,0.280038,0.915000,0.948956,0.866525,0.905869,0.974901
2000,0.309700,0.183234,0.931000,0.931478,0.921610,0.926518,0.983593
2500,0.229600,0.560935,0.875000,0.800693,0.978814,0.880839,0.978954
3000,0.212700,0.707147,0.874000,0.792230,0.993644,0.881579,0.979903
3500,0.183800,0.352786,0.927000,0.887379,0.968220,0.926039,0.985958
4000,0.181400,0.164660,0.954000,0.973333,0.927966,0.950108,0.992906
4500,0.187900,0.396157,0.911000,0.968215,0.838983,0.898978,0.981931
5000,0.112400,0.174626,0.960000,0.965517,0.949153,0.957265,0.993568


TrainOutput(global_step=15750, training_loss=0.11261962550980527, metrics={'train_runtime': 1881.7554, 'train_samples_per_second': 33.479, 'train_steps_per_second': 8.37, 'total_flos': 2.86546329216e+17, 'train_loss': 0.11261962550980527, 'epoch': 7.0})

<h1>Evaluation</h1>

In [26]:
from tqdm import tqdm

In [27]:
# Ensure model is in evaluation mode
model.eval()

# Prepare a list to store submission results
submission_results = []

# Process test dataset
for sample in tqdm(test_ds):
    sample_id = sample["id"]  # Preserve original ID
    audio_input = sample["audio"]["array"]

    # Extract features using feature_extractor
    inputs = feature_extractor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True)

    # Move inputs to the correct device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run inference
    with torch.no_grad():
        logits = model(**inputs).logits  # Get raw model outputs (logits)

    # Convert logits to probabilities
    probabilities = softmax(logits.cpu().numpy(), axis=-1)

    # Probability of being synthetic (is_tts = 1), rounded to 3 decimal places
    is_tts_prob = round(probabilities[0, 1], 3)

    # Append result (id, is_tts probability)
    submission_results.append([sample_id, is_tts_prob])

100%|██████████| 2635/2635 [00:56<00:00, 46.51it/s]


<h1>Submission</h1>

In [28]:
# Convert to DataFrame (keeping only "id" and "is_tts")
submission_df = pd.DataFrame(submission_results, columns=["id", "is_tts"])
submission_df.head()

,id,is_tts
0,ASM_F_ANGER_00109,0.0
1,ASM_F_ANGER_00127,1.0
2,ASM_F_ANGER_00386,0.0
3,ASM_F_ANGER_00103,1.0
4,ASM_F_ANGER_00434,0.0


In [29]:
# Save to CSV without modifying id
submission_df.to_csv("./submission.csv", index=False)
print("Submission file saved correctly!")

Submission file saved correctly!
